# Training using unmodified Gaia DR2 parallax and no offset model

We use `ApogeeBCNN` model here

In [ ]:
import h5py
import numpy as np
import astropy.units as u

from astroNN.nn.losses import mean_absolute_error, mean_error, mean_absolute_percentage_error
from astroNN.models import ApogeeBCNN
from astroNN.nn.callbacks import ErrorOnNaN
from astroNN.gaia import mag_to_fakemag

with h5py.File('gaia_dr2_train.h5') as F:  # ensure the file will be cleaned up
    parallax = np.array(F['parallax'])
    parallax_error = np.array(F['parallax_err'])
    spectra = np.array(F['spectra'])
    Kcorr = np.array(F['corrected_K'])  # extinction corrected Ks
        
fakemag, fakemag_err = mag_to_fakemag(Kcorr, parallax*u.mas, parallax_error)

bcnn_net = ApogeeBCNN()
bcnn_net.num_hidden = [164, 64]
bcnn_net.max_epochs = 40
bcnn_net.targetname = ['Ks-band fakemag']
bcnn_net.callbacks = ErrorOnNaN()
bcnn_net.labels_norm_mode = 4  # scale label but not center it
bcnn_net._last_layer_activation = 'softplus'  # transform NN prediction to positive number only
bcnn_net.metrics = [mean_absolute_error, mean_error, mean_absolute_percentage_error]
bcnn_net.train(spectra, fakemag, labels_err=fakemag_err)
bcnn_net.save("astroNN_no_offset_model")

# Training using unmodified Gaia DR2 parallax and a constant offset model

We use `ApogeeDR14GaiaDR2BCNN` model here

In [ ]:
import h5py
import numpy as np

from astroNN.nn.losses import mean_absolute_error, mean_error, mean_absolute_percentage_error
from astroNN.models import ApogeeDR14GaiaDR2BCNN
from astroNN.nn.callbacks import ErrorOnNaN

with h5py.File('gaia_dr2_train.h5') as F:  # ensure the file will be cleaned up
    parallax = np.array(F['parallax'])
    parallax_error = np.array(F['parallax_err'])
    spectra = np.array(F['spectra'])
    Kcorr = np.array(F['corrected_K'])  # extinction corrected Ks

# since constant offset, give offset model featureless vectors
aux_data = np.ones((parallax.shape[0], 3))

# stack all data to the model (spectra, Ks_mag, whatever_to_model)
training_data_final = np.hstack((spectra, np.expand_dims(Kcorr, axis=1), aux_data))

bcnn_net = ApogeeDR14GaiaDR2BCNN()
bcnn_net.max_epochs = 40
bcnn_net.callbacks = ErrorOnNaN()
bcnn_net.labels_norm_mode = 4  # scale label but not center it
bcnn_net.num_hidden = [164, 64, 32, 16]
bcnn_net.metrics = [mean_absolute_error, mean_error, mean_absolute_percentage_error]
bcnn_net.train(training_data_final, parallax, labels_err=parallax_error)
bcnn_net.save("astroNN_constant_model")

# Training using unmodified Gaia DR2 parallax and a offset model depending several possible parameters

We use `ApogeeDR14GaiaDR2BCNN` model here

In [ ]:
from astroNN.nn.losses import mean_absolute_error, mean_error, mean_absolute_percentage_error
from astroNN.models import ApogeeDR14GaiaDR2BCNN
from astroNN.nn.callbacks import ErrorOnNaN
from astroNN.gaia import mag_to_fakemag
import h5py
import numpy as np
import astropy.units as u
from astroNN.nn.numpy import median_absolute_percentage_error as mape

with h5py.File('gaia_dr2_train.h5') as F:  # ensure the file will be cleaned up
    parallax = np.array(F['parallax'])
    parallax_error = np.array(F['parallax_err'])
    spectra = np.array(F['spectra'])
    Kcorr = np.array(F['corrected_K'])
    bp_rp = np.array(F['bp_rp'])
    phot_g_mean_mag = np.array(F['phot_g_mean_mag'])
    
    good_idx = ((bp_rp != -9999.) & (phot_g_mean_mag != -9999.))

    parallax = parallax[good_idx]
    parallax_error = parallax_error[good_idx]
    spectra = spectra[good_idx]
    Kcorr = Kcorr[good_idx]
    bp_rp = bp_rp[good_idx]
    phot_g_mean_mag = phot_g_mean_mag[good_idx]

# need to manually 
phot_g_mean_mag_std = np.std(phot_g_mean_mag)
phot_g_mean_mag_mean = np.mean(phot_g_mean_mag)
norm_phot_g_mean_mag = (phot_g_mean_mag - phot_g_mean_mag_mean) / phot_g_mean_mag_std
bp_rp_std = np.std(bp_rp)
bp_rp_mean = np.mean(bp_rp)
norm_bp_rp = (bp_rp - bp_rp_mean) / bp_rp_std
aux_data = np.stack((norm_phot_g_mean_mag, norm_bp_rp)).T

# stack all data to the model (spectra, Ks_mag, whatever_to_model)
training_data_final = np.hstack((spectra, np.expand_dims(Kcorr, axis=1), aux_data))

bcnn_net = ApogeeDR14GaiaDR2BCNN()
bcnn_net.max_epochs = 40
bcnn_net.callbacks = ErrorOnNaN()
bcnn_net.labels_norm_mode = 4  # scale label but not center it
bcnn_net.num_hidden = [164, 64, 32, 16]
bcnn_net.metrics = [mean_absolute_error, mean_error, mean_absolute_percentage_error]
bcnn_net.train(training_data_final, parallax, labels_err=parallax_error)
bcnn_net.save("astroNN_multivariate_model")